In [1]:
import os

from gensim.models import CoherenceModel

os.environ['KERAS_BACKEND'] = "torch"

## Kickstarter Less Dataset (256k)


In [3]:
from train import ABAEModelConfiguration
from core.train import ABAEModelManager

config = ABAEModelConfiguration(
    corpus_file="../data/processed-dataset/full/256k.preprocessed.csv",
    model_name="256k.full", aspect_size=16
)

manager = ABAEModelManager(config)

Pandas Apply:   0%|          | 0/288429 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/288429 [00:00<?, ?it/s]

INFO:gensim.utils:loading Word2Vec object from output\64k.full\embeddings.keras
DEBUG:smart_open.smart_open_lib:{'uri': 'output\\64k.full\\embeddings.keras', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'compression': 'infer_from_extension', 'transport_params': None}
INFO:gensim.utils:loading wv recursively from output\64k.full\embeddings.keras.wv.* with mmap=None
INFO:gensim.utils:setting ignored attribute cum_table to None
INFO:gensim.utils:Word2Vec lifecycle event {'fname': 'output\\64k.full\\embeddings.keras', 'datetime': '2025-01-03T14:59:44.610187', 'gensim': '4.3.3', 'python': '3.12.6 (tags/v3.12.6:a4a2d2b, Sep  6 2024, 20:11:23) [MSC v.1940 64 bit (AMD64)]', 'platform': 'Windows-11-10.0.26100-SP0', 'event': 'loaded'}


In [4]:
inference_model = manager.__prepare_evaluation_model()

D:\PycharmProjects\nlp-course-project\core\layer.py:126: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super(WeightedAspectEmb, self).__init__(**kwargs)


## Top words for each aspect

In [5]:
from core.evaluation import normalize
from core.evaluation import get_aspect_top_k_words

word_emb = normalize(inference_model.get_layer('word_embedding').weights[0].value.data)
aspect_embeddings = normalize(inference_model.get_layer('weighted_aspect_emb').W)

print(f"Word embeddings shape: {word_emb.shape}")
inv_vocab = manager.embedding_model.model.wv.index_to_key

aspects_top_k_words = [get_aspect_top_k_words(a, word_emb, inv_vocab) for a in aspect_embeddings]

Word embeddings shape: torch.Size([17278, 128])

Given aspect most representative words are:
Word:  thick (0.5558367967605591)
Word:  cardstock (0.5345015525817871)
Word:  organizer (0.5092227458953857)
Word:  flimsy (0.4967082142829895)
Word:  inlay (0.4964747428894043)
Word:  sturdy (0.4959457218647003)
Word:  durable (0.4943707287311554)
Word:  cardboard (0.49310219287872314)
Word:  etsy (0.48853981494903564)
Word:  acrylic (0.4873029887676239)
Word:  insert (0.48672184348106384)
Word:  gametrayz (0.48402220010757446)
Word:  wash (0.467433363199234)
Word:  glass (0.46160924434661865)
Word:  tarot (0.4553457498550415)
Word:  components- (0.4511100649833679)
Word:  matte (0.4478016495704651)
Word:  lid (0.44744208455085754)
Word:  misprint (0.44702038168907166)
Word:  font (0.44319209456443787)
Word:  plastic (0.4425453841686249)
Word:  pastel (0.4382006525993347)
Word:  vincent (0.4335036277770996)
Word:  linen (0.42840415239334106)
Word:  wooden (0.4270378649234772)

Given aspect mo

![title](../resources/umass.png)


In [6]:
import dataset

vocab = manager.embedding_model.model.wv.key_to_index
ds = dataset.PositiveNegativeCommentGeneratorDataset(config.corpus_file, vocab, 15)

Loading dataset from file: ../data/processed-dataset/full/256k.preprocessed.csv
Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/288429 [00:00<?, ?it/s]

Max sequence length calculation in progress...
Max sequence length is:  262 . The limit is set to 80 tokens.
We loose information on 65 points.This is 0.022535875380076207% of the dataset.
Padding sequences to length (80).


4.4. Word2vec Coherence Score
[https://www.baeldung.com/cs/topic-modeling-coherence-score#4-word2vec-coherence-score

In [8]:
from core.evaluation import coherence_per_aspect

aspect_words = [[word[0] for word in aspect] for aspect in aspects_top_k_words]
coherence_per_aspect(aspect_words, ds.text_ds, 5)

INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 1000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 2000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 3000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 4000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 5000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 6000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 7000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 8000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 9000 documents
INFO:gensim.topic_coherence.text_analysis:CorpusAccumulator accumulated stats from 10000 documents
INFO:gensim.topic_c

([-13.155683376032698,
  -17.515514821201542,
  -17.931916180562855,
  -17.809937006249978,
  -9.872873095775873,
  -15.049084453227668,
  -14.6809609127482,
  -12.182438562750617,
  -14.287589739595905,
  -11.753537667572562,
  -13.738955325596555,
  -18.047824322014463,
  -11.357549457884954,
  -16.272808119015277,
  -16.182428021407052,
  -12.86136718040405],
 <gensim.models.coherencemodel.CoherenceModel at 0x219d0571520>)

In [ ]:
# todo: graph of the loss of coherence compared to word considered increase